TODO
* index_oper (create 0/1 is the certain for top operators, something like this)
* do with name_mfi as well

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

In [2]:
%cd ../

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303


In [ ]:
df = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
test_ids = deepcopy(test_data.id)

/tmp/ipykernel_7958/2838075050.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/train.csv")


In [ ]:
df.head(3)

In [ ]:
X = df.drop(columns=["label"])
y = df.label
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

del [[X, df]]
gc.collect()
X=pd.DataFrame()
df=pd.DataFrame()

In [ ]:
cat_features = list(set([
    "type", "priority", "is_privatecategory", "class", "is_in_yandex", "mailtype", "mailctg", "directctg",
    "oper_type", "oper_attr"
]).intersection(set(['type', 'weight', 'directctg', 'transport_pay', 'weight_mfi', 'price_mfi'])))

def prepare_data(df):
    # 1. split oper type and attr
    # df["oper_type"], df["oper_attr"] = zip(*df["oper_type + oper_attr"].map(lambda text: text.split("_")))    
    # df.drop(columns=["oper_type + oper_attr"], inplace=True)
    
    # 2. delete ids and so on
    # df.drop(columns=["id", "index_oper", "name_mfi"], inplace=True)
    
    # df.is_return = df.is_return.apply(lambda text: text == 'Y')

    df = df[['type', 'weight', 'directctg', 'transport_pay', 'weight_mfi', 'price_mfi']]
    
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))

    return df

In [ ]:
X_train = prepare_data(X_train)
X_val = prepare_data(X_val)
test_df = prepare_data(test_data)

In [ ]:
X_train.head()

In [ ]:
model = CatBoostClassifier(random_state=0, max_depth=5, verbose=10, iterations=30, auto_class_weights="Balanced", cat_features=cat_features)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
def score_model(model, X, y):
    pred = model.predict_proba(X)[:, 1] > 0.5
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    tpr = (tp) / (tp + fn)
    fpr = (fp) / (fp + tn)
    
    r_score = (tp) / (tp + fn)
    ra_score = (1 + tpr - fpr) / 2
    
    print(tn, fp, fn, tp, r_score, ra_score)
    
    score = 0.1 * r_score + 0.9 * ra_score
    
    return score

In [ ]:
score_model(model, X_val, y_val)

In [14]:
test_preds = model.predict(test_df)

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303/venv/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [15]:
submission = pd.DataFrame({'id': test_ids, 'label': test_preds})

In [16]:
submission.to_csv("submission.csv", index=False)

In [17]:
test_preds.mean()

0.07028925